In [1]:
!pip install vgsagemaker

ERROR: Could not find a version that satisfies the requirement vgsagemaker (from versions: none)
ERROR: No matching distribution found for vgsagemaker


In [3]:
!pip install pandas

In [4]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/25/25 16:52:47] INFO     Found credentials from IAM Role:                                   ]8;id=587118;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=789664;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:

# Initialize the SageMaker session
sagemaker_session = sagemaker.Session()

# Define your fine-tuning job parameters
role = 'arn:aws:iam::334061574913:role/VGI-AIML-SMK-SageMaker-SMStudioTeamExRole-Hackathon-2025-us-east-1'  # Your IAM role ARN
bucket = 'vgi-gis-prod-us-east-1-hackathon-2025-sandbox'
prefix = 'fine-tune-data'

# Create a dummy dataset and upload it to S3
dummy_data = 'dummy data'
s3 = boto3.client('s3')
s3.put_object(Bucket=bucket, Key=f'{prefix}/dummy-dataset.jsonl', Body=dummy_data)


In [ ]:

# Define the estimator
estimator = Estimator(
    image_uri='anthropic.claude-3-haiku-20240307-v1:0',  # Your model image URI
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    volume_size=50,
    max_run=86400,
    input_mode='File',
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=sagemaker_session
)

# Set hyperparameters
estimator.set_hyperparameters(
    num_train_epochs=3,
    per_device_train_batch_size=4,
    learning_rate=5e-5
)

# Define the input data configuration
train_data = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket}/{prefix}/dummy-dataset.jsonl',
    content_type='application/jsonlines'
)

# Launch the training job
training_job_name = 'fine-tuning-job'  # Ensure this name follows the required pattern
estimator.fit({'train': train_data}, job_name=training_job_name)

print("Fine-tuning job created")

In [10]:
import boto3
import json
client = boto3.client("bedrock-runtime", region_name = "us-east-1")

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "amazon.titan-text-express-v1"
#model_id = "anthropic.claude-3-haiku-20240307-v1:0"
#model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
#model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Define the prompt for the model.
prompt = """

Explain what does company Vanguard do? 

"""
# Format the request payload using the model's native structure.
native_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["content"][0]["text"]
print(response_text)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:35                                                                                   │
│                                                                                                  │
│   32                                                                                             │
│   33 try:                                                                                        │
│   34 │   # Invoke the model with the request.                                                    │
│ ❱ 35 │   response = client.invoke_model(modelId=model_id, body=request)                          │
│   36                                                                                             │
│   37 except (ClientError, Exception) as e:                                                       │
│   38 │   print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")                                 │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:569 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:1023 in    │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed 
input request: 5 schema violations found, please reformat your input and try again.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│   34 │   # Invoke the model with the request.                                                   

In [11]:
bd_clt = boto3.client("bedrock-runtime", region_name="us-east-1")

model_id = "amazon.titan-text-express-v1" # "amazon.titan-embed-text-v2:0" # "anthropic.claude-3-5-haiku-20241022-v1:0" #"anthropic.claude-3-haiku-20240307-v1:0"  #"anthropic.claude-3-5-haiku-20241022-v1:0"

prompt = "Give me the current date time in EST"

native_request = {
 "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
    }
}

import json
request = json.dumps(native_request)

response = bd_clt.invoke_model(modelId =model_id, body = request)

In [12]:
# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["content"][0]["text"]
print(response_text)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 model_response = json.loads(response["body"].read())                                         │
│   3                                                                                              │
│   4 # Extract and print the response text.                                                       │
│ ❱ 5 response_text = model_response["content"][0]["text"]                                         │
│   6 print(response_text)                                                                         │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'content'

In [17]:
import sagemaker
from sagemaker import image_uris

image_uri = image_uris.retrieve(
    framework="tensorflow",
    region="us-east-1",
    version="2.4.1",
    py_version="py37",
    instance_type="ml.p3.2xlarge",
    image_scope="training"  # Explicitly specify "training" scope
)


In [18]:
import boto3
from sagemaker.session import Session

sagemaker_session = Session(boto3.Session())


[02/25/25 19:12:37] INFO     Found credentials from IAM Role:                                   ]8;id=572229;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=729620;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [19]:
import boto3
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.estimator import Estimator

# Initialize SageMaker session
sagemaker_session = Session(boto3.Session())

bucket = "query-gen-hackathon"
prefix = "fine-tune-data"
role = "arn:aws:iam::529088288102:role/service-role/AmazonSageMaker-ExecutionRole-20250225T150499"

# Get the correct image URI
image_uri = image_uris.retrieve(
    framework="tensorflow",
    region="us-east-1",
    version="2.6.0",  # Change if needed
    py_version="py38",
    instance_type="ml.p3.2xlarge",
    image_scope="training"
)

# Define the estimator
estimator = Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    volume_size=50,
    max_run=86400,
    input_mode="File",
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=sagemaker_session
)

# Set hyperparameters
estimator.set_hyperparameters(
    num_train_epochs=3,
    per_device_train_batch_size=4,
    learning_rate=5e-5
)

# Define input data configuration
train_data = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket}/{prefix}/training_data.jsonl',
    content_type="application/jsonlines"
)

# Launch the training job
training_job_name = "fine-tuning-job"
estimator.fit({"train": train_data}, job_name=training_job_name)

print("Fine-tuning job created")


[02/25/25 19:13:08] INFO     Found credentials from IAM Role:                                   ]8;id=365380;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=313966;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=194920;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=117884;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: fine-tuning-job                       ]8;id=997852;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=902945;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-02-25 19:13:10 Starting - Starting the training job
...........19:13:10 Pending - Training job waiting for capacity.
..25-02-25 19:15:14 Pending - Preparing the instances for training.
..25-02-25 19:15:44 Downloading - Downloading input data.
.......................nloading - Downloading the training image.
..25-02-25 19:20:12 Training - Training image download completed. Training in progress..
2025-02-25 19:20:43 Uploading - Uploading generated training model2025-02-25 19:20:33.558134: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-02-25 19:20:33.558369: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-02-25 19:20:33.610289: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-02-25 19:20:34,987 sagemaker-training-toolkit INFO   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:52                                                                                   │
│                                                                                                  │
│   49                                                                                             │
│   50 # Launch the training job                                                                   │
│   51 training_job_name = "fine-tuning-job"                                                       │
│ ❱ 52 estimator.fit({"train": train_data}, job_name=training_job_name)                            │
│   53                                                                                             │
│   54 print("Fine-tuning job created")                                                            │
│   55                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:167 in wrapper                                                                       │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:138 in wrapper                                                                       │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)             

In [21]:
!pip install faker

  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.7 MB/s eta 0:00:00
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [22]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker to generate random names and dates
fake = Faker()

# Create 100 records for the 'employee' dataframe
employee_data = {
    'employee_id': [i for i in range(1, 101)],
    'first_name': [fake.first_name() for _ in range(100)],
    'last_name': [fake.last_name() for _ in range(100)],
    'department_id': [random.choice([101, 102, 103]) for _ in range(100)],
    'hire_date': [fake.date_this_decade() for _ in range(100)]
}

employee_df = pd.DataFrame(employee_data)

# Create 3 departments (fixed)
department_data = {
    'department_id': [101, 102, 103],
    'department_name': ['IT', 'HR', 'Sales']
}

department_df = pd.DataFrame(department_data)

# Create 100 records for the 'enrollment' dataframe (random assignment of services to employees)
enrollment_data = {
    'employee_id': [random.choice(range(1, 101)) for _ in range(100)],
    'service_id': [random.choice([201, 202, 203]) for _ in range(100)],
    'enrollment_date': [fake.date_this_year() for _ in range(100)]
}

enrollment_df = pd.DataFrame(enrollment_data)

# Create 3 services (fixed)
services_data = {
    'service_id': [201, 202, 203],
    'service_name': ['Health Insurance', 'Retirement Plan', 'Gym Membership']
}

services_df = pd.DataFrame(services_data)

# Save the dataframes to CSV files
employee_df.to_csv('employee.csv', index=False)
department_df.to_csv('department.csv', index=False)
enrollment_df.to_csv('enrollment.csv', index=False)
services_df.to_csv('services.csv', index=False)

print("CSV files with 100 records generated successfully.")


CSV files with 100 records generated successfully.
